### UnemploymentRate Data Process from Jan-2020 to Oct-2020

Use Pandas to import two files consisting of raw data spanning from August 2019 to September 2020 and preleminary data for October 2020 (downloaded from [Bureau of Labor Statistics](https://www.bls.gov/lau/#tables )) into DataFrame objects.




In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pylab as plt
from scipy.interpolate import interp1d

In [ ]:
### UnemploymentRate, from Jan-2020 to Sep-2020
df1 = pd.read_excel('./data/unemploymentRate.xlsx')   ### file 1
df_1 = df1[['Unnamed: 1','Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 7', 'Unnamed: 8' ]].drop([0,1,2,3,4])
df_1.columns = ['State Code','State Abbreviation','Period','Labor Force','Unemployed','Unemployment Rate']

In [ ]:
### UnemploymentRate, Oct-2020
df2 = pd.read_excel ('./data/unemploymentRate_Oct.xlsx')    ### file 2
df_2 = df2[['State and area', 'Unnamed: 4', 'Unnamed: 8', 'Unnamed: 12']].drop([0,1,2])
df_2.columns =['State Abbreviation','Labor Force','Unemployed','Unemployment Rate']

Define a function that takes a state code and returns the DataFrame object containing the semimonthly data of labor force, unemployment and unemployment rate of the state. The function aggregates the state labor force and unemployment data from Jan-20 to Sep-20 and calculates the state unemployment rate. The function obtains the state labor force and unemployment data, and unemployment rate in Oct-20 from the data file. The semimonthly data is obtained via linear interpolation of the monthly data.

In [ ]:
date = ["Jan-20", "Feb-20", "Mar-20", "Apr-20", "May-20", "Jun-20", "Jul-20", "Aug-20", "Sep-20 p"]
date_new = ["Jan-20", "Feb-20", "Mar-20", "Apr-20", "May-20", "Jun-20", "Jul-20", "Aug-20", "Sep-20", "Oct-20"] ### oct-20 in second csv file
state_list = {'06':'California', '36':'New York', '12':'Florida', '42':'Pennsylvania', '55':'Wisconsin', '13':'Georgia', '48':'Texas'}
my_month = [1, 1, 2, 2 ,3 ,3 ,4 ,4 ,5 ,5 ,6, 6, 7, 7, 8, 8, 9, 9, 10,]
my_segment = ['1-1', '1-2', '2-1', '2-2' ,'3-1' ,'3-2' ,'4-1' ,'4-2' ,'5-1' ,'5-2' ,'6-1', '6-2', '7-1', '7-2', '8-1', '8-2', '9-1', '9-2', '10-1',]

x = np.linspace(1, 10, num=10, endpoint=True)
xnew = np.linspace(1, 10, num=19, endpoint=True)  ### build new dataframe (new period to half month), 19 rows for each state

In [ ]:
############  function: clean data for each state, 
############  put all needed data into a new dataframe

def states_rate(state_code):
    ### de_new: put 7 states data into df_new dataframe
    df_new = df_1.loc[df_1['State Code'] == state_code]  
    ### df_states : build a new dataframe, put data in the new dataframe,
    ###             keep only three columns
    df_states = pd.DataFrame(np.zeros((10,3)), columns = [ "Labor Force", "Unemployed", "Unemployment Rate (%)"])   
    
    state = [state_list[state_code]]*len(xnew)  ### state: output the column "State", e.g. california 
    df_states_new = pd.DataFrame()
    
    for line in range(len(df_new)):
        for counter in range(len(date)):
            if df_new.iloc[line]["Period"] == date[counter]:  ### if "Period" column is Jan, Feb, etc
                df_states["Labor Force"][counter] += df_new.iloc[line]["Labor Force"]    ### sum the column "Labor Force"
                df_states["Unemployed"][counter] += df_new.iloc[line]["Unemployed"]      ### sum the column "Unemployed"
    
    df_oct_states = df_2.loc[df_2['State Abbreviation'] == state_list[state_code]]
    df_states["Labor Force"][9] = df_oct_states["Labor Force"]
    df_states["Unemployed"][9] =df_oct_states["Unemployed"]
   
    for counter in range(len(date_new)):   ### get unemployment rate each month
        df_states["Unemployment Rate (%)"][counter] = round(df_states["Unemployed"][counter]/df_states["Labor Force"][counter]*100, 2)
    #df_states["States"] = state
    #df_states["Period"] = date_new
    for header in df_states.iloc[:,0:3].keys():  
        y = df_states[header]
        f = interp1d(x, y)
        fnew = f(xnew)
        df_states_new[header] = fnew

    df_states_new["States"] = state
    df_states_new['month'] = my_month
    df_states_new["Segment"] = my_segment
    return df_states_new

In [ ]:
df_unemployment = pd.concat([states_rate("06"),states_rate("55"),states_rate("36"), states_rate("12"),
           states_rate("42"),states_rate("13"), states_rate("48")], axis=0).reset_index().drop(['index'], axis = 1)

In [ ]:
df_unemployment = df_unemployment[['States', 'Labor Force', 'Unemployed', 'Unemployment Rate (%)', 'Segment', 'month']]
df_unemployment

,States,Labor Force,Unemployed,Unemployment Rate (%),Segment,month
0,California,19477354.0,839987.0,4.310,1-1,1
1,California,19502919.5,841174.0,4.310,1-2,1
2,California,19528485.0,842361.0,4.310,2-1,2
3,California,19360301.0,974383.5,5.035,2-2,2
4,California,19192117.0,1106406.0,5.760,3-1,3
...,...,...,...,...,...,...
128,Texas,14330373.0,1000259.0,6.980,8-1,8
129,Texas,14269358.5,1088924.0,7.635,8-2,8
130,Texas,14208344.0,1177589.0,8.290,9-1,9
131,Texas,14151318.0,1063940.5,7.515,9-2,9


### Polling Data Process from Jan-2020 to Oct-2020

*   List item
*   List item



Used Pandas library to read csv source file 'president_polls.csv that downloaded from FiveThirtyEight website, please save this source file in the same directory, folder named 'Election_preference_polls_data' specified parse_dates argument for easy handle of dates of column 'start_date' and 'end_date'. Imported warning module to ignore warnings.


In [ ]:
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv("./data/president_polls.csv", parse_dates = ["start_date", "end_date"])
#data.head()

Parsed data and only retain colunmns of 'state', 'start_date'(start date of a pollster data), 'answer'(each candidates), and 'pct'(percentage of each candidates).

In [ ]:
data_refined = data[["state","start_date","answer","pct"]]
#data_refined.head()

Drop any non-value cells by .dropna().

In [ ]:
data_refined = data_refined.dropna()

Restricted column of state to only retain seven key states that we are focusing on and filtered candidates from column 'answer' to only Trump and Biden.

In [ ]:
answer_lst = ["Biden", "Trump"]
state_lst = ["Pennsylvania", "New York", "Florida", "California", "Georgia", "Texas", "Wisconsin"]
data_refined = data_refined[data_refined["answer"].isin(answer_lst)]
data_refined = data_refined[data_refined["state"].isin(state_lst)]

Used pandas Series.dt.year which is the year of the datetime module to filter the column 'start_date' to only retain year is 2020.

In [ ]:
data_refined = data_refined[(data_refined['start_date'].dt.year == 2020)]
#data_refined.shape
#data_refined.head()

For easy handling of data, seperated dataframe into two, one is where 'answer' column is Trump and another one is Biden. Sorted values using .sort_values() by 'state' column first, then the 'start_date' column. 

In [ ]:
data_Trump = data_refined[(data_refined['answer']=='Trump')]
data_Biden = data_refined[(data_refined['answer']=='Biden')]

In [ ]:
data_Trump.sort_values(by=['state','start_date'], inplace=True)

In [ ]:
data_Biden.sort_values(by=['state','start_date'], inplace=True)

Created columns of 'year', 'month', 'day' by datetime methods. Renamed 'pct' column to 'Trump(%)' and 'Biden(%)'  respectively. Dropped column 'answer' and 'start_date'.

In [ ]:
data_Trump['year'] = data_Trump['start_date'].dt.year
data_Trump['month'] = data_Trump['start_date'].dt.month
data_Trump['day'] = data_Trump['start_date'].dt.day
data_Trump = data_Trump.rename(columns = {"pct": "Trump(%)"})
data_Trump = data_Trump.drop(columns = ["answer", "start_date"])
#data_Trump.head()

In [ ]:
data_Biden['year'] = data_Biden['start_date'].dt.year
data_Biden['month'] = data_Biden['start_date'].dt.month
data_Biden['day'] = data_Biden['start_date'].dt.day
data_Biden = data_Biden.rename(columns = {"pct": "Biden(%)"})
data_Biden = data_Biden.drop(columns = ["answer", "start_date"])

#data_Biden.head()

Created a column of 'Segment' which describes whether each row's day fall in first half of month or second half of month. segment is denoted like '1-1' for first half of January and '1-2' denotes second half of Janaury. 

In [ ]:
segment = []
for day in data_Trump["day"]:
    if day <= 15:
        segment.append("-1")
    else:
        segment.append("-2")
data_Trump['segment'] = segment
#data_Trump.head()   

In [ ]:
segment = []
for day in data_Biden["day"]:
    if day <= 15:
        segment.append("-1")
    else:
        segment.append("-2")
data_Biden['segment'] = segment
#data_Biden.head()

In [ ]:
data_Trump['Segment'] = data_Trump['month'].astype(str) + data_Trump['segment']
data_Trump = data_Trump.drop(['segment','day'], axis = 1)
#data_Trump.head()

In [ ]:
data_Biden['Segment'] = data_Biden['month'].astype(str) + data_Biden['segment']
data_Biden = data_Biden.drop(['segment','day'], axis = 1)
#data_Biden.head()

Seperated data_Trump and data_Biden each into seven sub-dataframe, and then for each sub-dataframe, used groupby on 'Segment' and compute the mean of each candidate's percentage. The mean of each corresponding segment is stored in the new column: 'Trump (%)', annd 'Biden (%)'. Note there is a space in this column name that differs from another column. Then merge the groupby object back to sub-dataframe. Now we have each candidate's dataframe seperate into seven states, and each segment, which is on half month base, has its own value. In another word, half month's average of percentage of each candidates in each states is calculated. 

In [ ]:
ca_Trump = data_Trump[(data_Trump['state'] == 'California')]
pa_Trump = data_Trump[(data_Trump['state'] == 'Pennsylvania')]
ny_Trump = data_Trump[(data_Trump['state'] == 'New York')]
fl_Trump = data_Trump[(data_Trump['state'] == 'Florida')]
ga_Trump = data_Trump[(data_Trump['state'] == 'Georgia')]
tx_Trump = data_Trump[(data_Trump['state'] == 'Texas')]
wi_Trump = data_Trump[(data_Trump['state'] == 'Wisconsin')]

ca_Biden = data_Biden[(data_Biden['state'] == 'California')]
pa_Biden = data_Biden[(data_Biden['state'] == 'Pennsylvania')]
ny_Biden = data_Biden[(data_Biden['state'] == 'New York')]
fl_Biden = data_Biden[(data_Biden['state'] == 'Florida')]
ga_Biden = data_Biden[(data_Biden['state'] == 'Georgia')]
tx_Biden = data_Biden[(data_Biden['state'] == 'Texas')]
wi_Biden = data_Biden[(data_Biden['state'] == 'Wisconsin')]
# ca_Biden.head(30)

In [ ]:
Trump_state_lst = [ca_Trump, pa_Trump, ny_Trump, fl_Trump, ga_Trump, tx_Trump, wi_Trump]
Biden_state_lst = [ca_Biden, pa_Biden, ny_Biden, fl_Biden, ga_Biden, tx_Biden, wi_Biden]

tavg_ca = ca_Trump.groupby('Segment')[['Trump(%)']].mean().rename(columns = {'Trump(%)':'Trump (%)'}) 
tavg_pa = pa_Trump.groupby('Segment')[['Trump(%)']].mean().rename(columns = {'Trump(%)':'Trump (%)'}) 
tavg_ny = ny_Trump.groupby('Segment')[['Trump(%)']].mean().rename(columns = {'Trump(%)':'Trump (%)'}) 
tavg_fl = fl_Trump.groupby('Segment')[['Trump(%)']].mean().rename(columns = {'Trump(%)':'Trump (%)'}) 
tavg_ga = ga_Trump.groupby('Segment')[['Trump(%)']].mean().rename(columns = {'Trump(%)':'Trump (%)'}) 
tavg_tx = tx_Trump.groupby('Segment')[['Trump(%)']].mean().rename(columns = {'Trump(%)':'Trump (%)'}) 
tavg_wi = wi_Trump.groupby('Segment')[['Trump(%)']].mean().rename(columns = {'Trump(%)':'Trump (%)'}) 

bavg_ca = ca_Biden.groupby('Segment')[['Biden(%)']].mean().rename(columns = {'Biden(%)':'Biden (%)'}) 
bavg_pa = pa_Biden.groupby('Segment')[['Biden(%)']].mean().rename(columns = {'Biden(%)':'Biden (%)'}) 
bavg_ny = ny_Biden.groupby('Segment')[['Biden(%)']].mean().rename(columns = {'Biden(%)':'Biden (%)'}) 
bavg_fl = fl_Biden.groupby('Segment')[['Biden(%)']].mean().rename(columns = {'Biden(%)':'Biden (%)'}) 
bavg_ga = ga_Biden.groupby('Segment')[['Biden(%)']].mean().rename(columns = {'Biden(%)':'Biden (%)'}) 
bavg_tx = tx_Biden.groupby('Segment')[['Biden(%)']].mean().rename(columns = {'Biden(%)':'Biden (%)'}) 
bavg_wi = wi_Biden.groupby('Segment')[['Biden(%)']].mean().rename(columns = {'Biden(%)':'Biden (%)'}) 


In [ ]:
ca_Trump = pd.merge(ca_Trump, tavg_ca, how = 'inner', on = 'Segment')  
pa_Trump = pd.merge(pa_Trump, tavg_pa, how = 'inner', on = 'Segment')
ny_Trump = pd.merge(ny_Trump, tavg_ny, how = 'inner', on = 'Segment')
fl_Trump = pd.merge(fl_Trump, tavg_fl, how = 'inner', on = 'Segment')
ga_Trump = pd.merge(ga_Trump, tavg_ga, how = 'inner', on = 'Segment')
tx_Trump = pd.merge(tx_Trump, tavg_tx, how = 'inner', on = 'Segment')
wi_Trump = pd.merge(wi_Trump, tavg_wi, how = 'inner', on = 'Segment')

ca_Biden = pd.merge(ca_Biden, bavg_ca, how = 'inner', on = 'Segment')  
pa_Biden = pd.merge(pa_Biden, bavg_pa, how = 'inner', on = 'Segment')
ny_Biden = pd.merge(ny_Biden, bavg_ny, how = 'inner', on = 'Segment')
fl_Biden = pd.merge(fl_Biden, bavg_fl, how = 'inner', on = 'Segment')
ga_Biden = pd.merge(ga_Biden, bavg_ga, how = 'inner', on = 'Segment')
tx_Biden = pd.merge(tx_Biden, bavg_tx, how = 'inner', on = 'Segment')
wi_Biden = pd.merge(wi_Biden, bavg_wi, how = 'inner', on = 'Segment')

for i in Trump_state_lst:
    i.sort_values(by=['state','month'], inplace=True)

for i in Biden_state_lst:
    i.sort_values(by=['state','month'], inplace=True)
ca_Biden.head(20)

,state,Biden(%),year,month,Segment,Biden (%)
0,California,59.00,2020,1,1-1,59.000
1,California,59.90,2020,2,2-1,58.450
2,California,57.00,2020,2,2-1,58.450
3,California,57.80,2020,2,2-2,59.725
4,California,60.00,2020,2,2-2,59.725
5,California,62.10,2020,2,2-2,59.725
6,California,59.00,2020,2,2-2,59.725
7,California,67.00,2020,3,3-2,67.000
8,California,65.17,2020,5,5-1,65.170
9,California,57.00,2020,5,5-2,57.500


In [ ]:
state_lst = ["Pennsylvania", "New York", "Florida", "California", "Georgia", "Texas", "Wisconsin"]

Deleted duplicates in the data set because of previous step. Concatenated each candidates in seven states back to one dataframe. Dropped previous each single date's percentage column,'Trump(%)' and 'Biden(%)'

In [ ]:
ca_Trump.drop_duplicates(subset = 'Segment', keep = "first", inplace = True)
pa_Trump.drop_duplicates(subset = 'Segment', keep = "first", inplace = True)
ny_Trump.drop_duplicates(subset = 'Segment', keep = "first", inplace = True)
fl_Trump.drop_duplicates(subset = 'Segment', keep = "first", inplace = True)
ga_Trump.drop_duplicates(subset = 'Segment', keep = "first", inplace = True)
tx_Trump.drop_duplicates(subset = 'Segment', keep = "first", inplace = True)
wi_Trump.drop_duplicates(subset = 'Segment', keep = "first", inplace = True)

ca_Biden.drop_duplicates(subset = 'Segment', keep = "first", inplace = True)
pa_Biden.drop_duplicates(subset = 'Segment', keep = "first", inplace = True)
ny_Biden.drop_duplicates(subset = 'Segment', keep = "first", inplace = True)
fl_Biden.drop_duplicates(subset = 'Segment', keep = "first", inplace = True)
ga_Biden.drop_duplicates(subset = 'Segment', keep = "first", inplace = True)
tx_Biden.drop_duplicates(subset = 'Segment', keep = "first", inplace = True)
wi_Biden.drop_duplicates(subset = 'Segment', keep = "first", inplace = True)


In [ ]:
frames_Biden = [ca_Biden,fl_Biden, ga_Biden, ny_Biden, pa_Biden, tx_Biden, wi_Biden]
frames_Trump = [ca_Trump,fl_Trump, ga_Trump, ny_Trump, pa_Trump, tx_Trump, wi_Trump]

Trump_df = pd.concat(frames_Trump)
Trump_df = Trump_df.drop('Trump(%)', axis = 1)   

Biden_df = pd.concat(frames_Biden)
Biden_df = Biden_df.drop('Biden(%)', axis = 1) 

Combine two dataframes of Trump's and Biden's into one dataframe, Adjust column names to aligh with unemployment rate dataframes and COVID19 dataframes.

In [ ]:
df_polling_data = pd.merge(Trump_df, Biden_df, how = 'outer') 
df_polling_data.rename(columns = {'state' : "States"}, inplace = True)
df_polling_data['year'] = df_polling_data['year'].astype(str)
df_polling_data

,States,year,month,Segment,Trump (%),Biden (%)
0,California,2020,1,1-1,37.000000,59.000000
1,California,2020,2,2-1,34.275000,58.450000
2,California,2020,2,2-2,31.577273,59.725000
3,California,2020,3,3-2,29.000000,67.000000
4,California,2020,5,5-1,34.830000,65.170000
...,...,...,...,...,...,...
113,Wisconsin,2020,8,8-2,42.334231,50.783077
114,Wisconsin,2020,9,9-1,43.417692,50.777308
115,Wisconsin,2020,9,9-2,42.942500,53.102500
116,Wisconsin,2020,10,10-1,43.668182,51.404545


### Covid_19 Data Process from Jan-2020 to Oct-2020

In [ ]:
from operator import itemgetter
import csv

In [ ]:
workpath = "./data/"
covid_frame = pd.read_csv("./data/3_cases_and_deaths_by_state_timeseries.csv")
#state_study = ["California", "New York", "Texas", "Georgia", "Florida", "Pennsylvania", "Wisconsin"]

In [ ]:
# input raw polls data as dataframe
covid_raw = covid_frame.values.tolist()

covid_daily_7states = []
for row in covid_raw:
    if row[0] in ["California", "New York", "Texas", "Georgia", "Florida", "Pennsylvania", "Wisconsin"]:
        date = row[1].split("-")  # [year, month, day]
        segment = 1 if int(date[2]) <= 15 else 2
        #print(row)
        new_case = 0 if row[7] == "" else float(row[7])
        new_case_1e6 = 0.0 if row[12] == "" else float(row[12])
        new_death = 0 if row[9] == "" else float(row[9])
        new_death_1e6 = 0.0 if row[11] == "" else float(row[11])
        covid_short = [row[0], date[0], date[1], segment, new_case, new_case_1e6, new_death, new_death_1e6]
        #print(covid_short)
        covid_daily_7states.append(covid_short)

covid_daily_7states = sorted(covid_daily_7states, key = itemgetter(0,1,2,3,))  
#covid_daily_7states[900:950]

In [ ]:
covid = covid_daily_7states[:]
covid_15d_7states = []

# initializing
sum_newcase = covid[0][4]
sum_newcase_1e6 = covid[0][5]
sum_newdeath = covid[0][6]
sum_newdeath_1e6 = covid[0][7]
count_day = 1
for i in range(1, len(covid)):
    if (covid[i][0] == covid[i-1][0]) and (covid[i][1] == covid[i-1][1]) and (covid[i][2] == covid[i-1][2]) and (covid[i][3] == covid[i-1][3]):
        sum_newcase += covid[i][4]
        sum_newcase_1e6 += covid[i][5]
        sum_newdeath += covid[i][6]
        sum_newdeath_1e6 += covid[i][7]
        count_day += 1
    else:
        #print(sum_newcase,sum_newcase_1e6,sum_newdeath,sum_newdeath_1e6,count_day)
        aver_newcase = float(sum_newcase / count_day)
        aver_newcase_1e6 = float(sum_newcase_1e6 / count_day)
        aver_newdeath = float(sum_newdeath / count_day)
        aver_newdeath_1e6 = float(sum_newdeath_1e6 / count_day)

        sum_newcase = covid[i][4]
        sum_newcase_1e6 = covid[i][5]
        sum_newdeath = covid[i][6]
        sum_newdeath_1e6 = covid[i][7]
        count_day = 1

        covid_15d = covid[i][0:4] + [aver_newcase, aver_newcase_1e6, aver_newdeath, aver_newdeath_1e6]
        #print(covid_15d)
        covid_15d_7states.append(covid_15d)

# seperate the data for each state
covid_15d_ca = []
covid_15d_ny = []
covid_15d_ga = []
covid_15d_tx = []
covid_15d_fl = []
covid_15d_pa = []
covid_15d_wi = []
for record in covid_15d_7states:
    if record[0] == "California":
        covid_15d_ca.append(record)
    elif record[0] == "New York":
        covid_15d_ny.append(record)
    elif record[0] == "Texas":
        covid_15d_tx.append(record)
    elif record[0] == "Georgia":
        covid_15d_ga.append(record)
    elif record[0] == "Florida":
        covid_15d_fl.append(record)
    elif record[0] == "Pennsylvania":
        covid_15d_pa.append(record)
    elif record[0] == "Wisconsin":
        covid_15d_wi.append(record)


In [ ]:
# output the 15d polls as a csv file for California
ca_file_15d = workpath + "covid_15d_California.csv"
with open(ca_file_15d, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in covid_15d_ca:
        csvwriter.writerow(row)

In [ ]:
ny_file_15d = workpath + "covid_15d_New York.csv"
with open(ny_file_15d, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in covid_15d_ny:
        csvwriter.writerow(row)

In [ ]:
tx_file_15d = workpath + "covid_15d_Texas.csv"
with open(tx_file_15d, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in covid_15d_tx:
        csvwriter.writerow(row)

In [ ]:
ga_file_15d = workpath + "covid_15d_Georgia.csv"
with open(ga_file_15d, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in covid_15d_ga:
        csvwriter.writerow(row)

In [ ]:
fl_file_15d = workpath + "covid_15d_Florida.csv"
with open(fl_file_15d, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in covid_15d_fl:
        csvwriter.writerow(row)

In [ ]:
pa_file_15d = workpath + "covid_15d_Pennsylvania.csv"
with open(pa_file_15d, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in covid_15d_pa:
        csvwriter.writerow(row)

In [ ]:
wi_file_15d = workpath + "covid_15d_Wisconsin.csv"
with open(wi_file_15d, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in covid_15d_wi:
        csvwriter.writerow(row)

In [ ]:
CA = pd.read_csv(workpath + "covid_15d_California.csv", names=['States', 'year', 'month', 'Segment',
                             'new_case', 'new_case_per_1e6', 'new_death',  'new_death_per_1e6'])
FL = pd.read_csv(workpath + "covid_15d_Florida.csv", names=['States', 'year', 'month', 'Segment',
                             'new_case', 'new_case_per_1e6', 'new_death',  'new_death_per_1e6'])
GA = pd.read_csv(workpath + "covid_15d_Georgia.csv", names=['States', 'year', 'month', 'Segment',
                             'new_case', 'new_case_per_1e6', 'new_death',  'new_death_per_1e6'])
NY = pd.read_csv(workpath + "covid_15d_New York.csv", names=['States', 'year', 'month', 'Segment',
                             'new_case', 'new_case_per_1e6', 'new_death',  'new_death_per_1e6'])
PA = pd.read_csv(workpath + "covid_15d_Pennsylvania.csv", names=['States', 'year', 'month', 'Segment',
                             'new_case', 'new_case_per_1e6', 'new_death',  'new_death_per_1e6'])
TX = pd.read_csv(workpath + "covid_15d_Texas.csv", names=['States', 'year', 'month', 'Segment',
                             'new_case', 'new_case_per_1e6', 'new_death',  'new_death_per_1e6'])
WI = pd.read_csv(workpath + "covid_15d_Wisconsin.csv", names=['States', 'year', 'month', 'Segment',
                             'new_case', 'new_case_per_1e6', 'new_death',  'new_death_per_1e6'])

In [ ]:
df_covid = pd.concat([CA, FL, GA, NY, PA, TX, WI], axis=0).reset_index().drop("index", axis = 1)
df_covid.rename(columns = {'state' : "States", 'segment': 'Segment'}, inplace = True)
df_covid

,States,year,month,Segment,new_case,new_case_per_1e6,new_death,new_death_per_1e6
0,California,2020,2,1,NaN,NaN,NaN,NaN
1,California,2020,2,2,0.333333,0.000667,0.000000,0.000000
2,California,2020,3,1,0.285714,0.000714,0.000000,0.000000
3,California,2020,3,2,24.066667,0.062000,0.333333,0.001333
4,California,2020,4,1,491.250000,1.241875,10.500000,0.027500
...,...,...,...,...,...,...,...,...
141,Wisconsin,2020,9,2,1046.733333,18.004000,6.133333,0.104667
142,Wisconsin,2020,10,1,2064.666667,35.515333,8.266667,0.141333
143,Wisconsin,2020,10,2,2670.066667,45.928667,19.533333,0.334667
144,Wisconsin,2020,11,1,4721.500000,81.213750,40.250000,0.692500


In [ ]:
df_covid["Segment"] = df_covid["month"].astype(str) + '-' + df_covid["Segment"].astype(str)
#df_covid

### Merge three dataset

Merge three dataset: Unemployment Rate data, Polling data and Covid_19 data. Perform the outer join on three columns(Segment, States, month)
First, merge df_unemployment and df_polling_data.

In [ ]:
##### df_polling_data and df_unemployment
df_merge_Polling_Unemployment = pd.merge(df_unemployment, df_polling_data, 
                         how = 'outer', on = ['Segment', 'States', 'month'])

#df_merge_Polling_Unemployment

Second, merge the third dataframe(df_covid). We will get a full dataframe with three datasets.

In [ ]:
######## merge three dataframe
df_merge_all = pd.merge(df_merge_Polling_Unemployment, df_covid, 
                         how = 'outer', on = ['States', 'month', 'Segment'])

df_merge_all.drop('year_y', axis = 1)
df_merge_all.rename(columns = {'year_x' : "year"}, inplace = True)

df_merge_all = df_merge_all[['States', 'year', 'month', 'Segment',
                             'Labor Force', 'Unemployed', 'Unemployment Rate (%)', 
                             'Trump (%)', 'Biden (%)',
                             'new_case', 'new_case_per_1e6', 'new_death',  'new_death_per_1e6']]

Change the column names, make it much easier to read. 

In [ ]:
##### format dataframe
df_merge_all.rename(columns = {'new_case' : "New Case", 'new_case_per_1e6' : 'New Case Per 10$^6$', 
                               'new_death' : 'New Death', 'new_death_per_1e6' : 'New Death Per 10$^6$'}, inplace = True)
df_merge_all['Trump (%)'] = round(df_merge_all['Trump (%)'] , 2)
df_merge_all['Biden (%)'] = round(df_merge_all['Biden (%)'] , 2)
df_merge_all['New Case'] = round(df_merge_all['New Case'] , 4)
df_merge_all['New Death'] = round(df_merge_all['New Death'] , 4)
df_merge_all['New Case Per 10$^6$'] = round(df_merge_all['New Case Per 10$^6$'] , 5)
df_merge_all['New Death Per 10$^6$'] = round(df_merge_all['New Death Per 10$^6$'] , 5)
df_merge_all

,States,year,month,Segment,Labor Force,Unemployed,Unemployment Rate (%),Trump (%),Biden (%),New Case,New Case Per 10$^6$,New Death,New Death Per 10$^6$
0,California,2020,1,1-1,19477354.0,839987.0,4.310,37.00,59.00,NaN,NaN,NaN,NaN
1,California,NaN,1,1-2,19502919.5,841174.0,4.310,NaN,NaN,NaN,NaN,NaN,NaN
2,California,2020,2,2-1,19528485.0,842361.0,4.310,34.28,58.45,NaN,NaN,NaN,NaN
3,California,2020,2,2-2,19360301.0,974383.5,5.035,31.58,59.72,0.3333,0.00067,0.0000,0.00000
4,California,NaN,3,3-1,19192117.0,1106406.0,5.760,NaN,NaN,0.2857,0.00071,0.0000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,Pennsylvania,NaN,11,11-2,NaN,NaN,NaN,NaN,NaN,3727.0000,29.10000,32.6667,0.25467
150,Texas,NaN,11,11-1,NaN,NaN,NaN,NaN,NaN,6059.9375,21.11375,73.6875,0.25688
151,Texas,NaN,11,11-2,NaN,NaN,NaN,NaN,NaN,8818.5333,30.72467,102.1333,0.35533
152,Wisconsin,NaN,11,11-1,NaN,NaN,NaN,NaN,NaN,4721.5000,81.21375,40.2500,0.69250


Write the dataframe to a csv file.

In [ ]:
##### write into a csv file
df_merge_all.to_csv("./data/final_merged_data.csv")